---

### **Project Overview**

You’re building your own **AI-powered knowledge base chatbot** that can answer questions using your personal or company data — things like PDFs, markdown files, and notes.
The model itself runs **locally** using **LM Studio**, and your **Streamlit** front-end provides a simple web chat interface.
Behind the scenes, the system uses **Retrieval-Augmented Generation (RAG)** — meaning it first searches your documents for relevant text, then feeds that into a local LLM to craft an intelligent, grounded response.

All free. All offline. All under your control.

---

### **Step-by-Step Roadmap**

**1. Environment Setup**

* Install **Python 3.10.10** and **CUDA 11.8**-compatible drivers.
* Create a new folder `rag-chatbot/` with subfolders: `app/`, `data/`, `models/`, `scripts/`.
* Add a `requirements.txt` file (from the previous step).
* Set up a virtual environment and install dependencies.

**2. LM Studio Configuration**

* Install LM Studio (open-source desktop app).
* Download a small local model (e.g., `Mistral-7B-Instruct` or `Llama-3-Instruct`).
* Enable the **local server API** inside LM Studio and note the port (usually `http://localhost:1234`).

**3. Data Ingestion + Embeddings**

* Place PDFs or markdowns inside `data/`.
* Use a `scripts/ingest_data.py` script to:

  * Read documents with `pypdf` or plain text loaders.
  * Generate embeddings using `sentence-transformers`.
  * Store them in a local **FAISS** vector index.

**4. RAG Pipeline Assembly**

* Build a retrieval pipeline with **LangChain**:

  * Use FAISS to search for the most relevant text chunks.
  * Send the top-ranked chunks and user question to LM Studio via HTTP.
  * Combine the retrieved info + model output for final answers.

**5. Streamlit Front-End**

* Create `app/app.py`:

  * Include a text input for user queries.
  * Display chat history (user + bot messages).
  * Connect to your RAG backend functions to fetch responses live.

**6. Testing and Tuning**

* Run `streamlit run app/app.py`.
* Ask domain questions — confirm that the chatbot uses your documents, not just general knowledge.
* Adjust chunk size, embedding model, or retrieval parameters for accuracy.

**7. Deployment (Optional)**

* Package as a standalone desktop app with **Streamlit** or run in a private LAN.
* Everything stays **offline and local**, respecting data privacy.

---

rag-chatbot/

├── app/               # Streamlit + RAG logic will go here later

├── data/              # Your PDFs / markdown / knowledge base files

├── models/            # (optional) local embeddings/models you might download

├── scripts/           # helper scripts we’ll write later

├── requirements.txt   # pinned versions (see below)

└── .env               # for LM Studio endpoint and secrets



In [1]:
# Project metadata / configuration
from pathlib import Path

PROJECT_CONFIG = {
    "project_name": "company_rag_chatbot",
    "description": "A RAG pipeline for internal knowledge base Q&A.",
    "data_dirs": {
        "policies": Path("data/policies"),
        "runbooks": Path("data/runbooks"),
        "faq": Path("data/faq"),
    },
    "embedding_model": "sentence-transformers/all-mpnet-base-v2",
    "vector_store": "faiss",
    "device_preference": "cuda",  # Will auto-detect GPU if available
}
PROJECT_CONFIG

{'project_name': 'company_rag_chatbot',
 'description': 'A RAG pipeline for internal knowledge base Q&A.',
 'data_dirs': {'policies': WindowsPath('data/policies'),
  'runbooks': WindowsPath('data/runbooks'),
  'faq': WindowsPath('data/faq')},
 'embedding_model': 'sentence-transformers/all-mpnet-base-v2',
 'vector_store': 'faiss',
 'device_preference': 'cuda'}

---
## Imports

Before building the RAG pipeline, we’ll import all required libraries.  
This section sets up the environment, verifies GPU availability, and ensures that we can use FAISS and Sentence Transformers efficiently.  

If a GPU is detected, embeddings and FAISS indexing will use CUDA for faster computation.  
Otherwise, it will automatically fall back to CPU mode.


In [ ]:
# Core imports
import os
import torch
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm

# NLP and Embedding models
from sentence_transformers import SentenceTransformer

# Vector store
import faiss

# LangChain for document loading and chunking
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import TextLoader, PyPDFLoader, UnstructuredFileLoader

# Check environment
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device.upper()}")

# Initialize embedding model
embedding_model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2", device=device)
print("Embedding model loaded successfully.")
